In [1]:
import numpy as np
from mindquantum.core.gates import RX, RY, RZ, H, X, Y, Z, CNOT
from mindquantum.core.circuit import Circuit
import mindspore as ms
from mindquantum.simulator import  Simulator
from mindquantum.core.gates import GroupedPauli
from mindquantum.core.operators import TimeEvolution,QubitOperator
from mindquantum.core.parameterresolver import PRConvertible,PRGenerator,ParameterResolver
from DQAS_tool import generate_pauli_string
from mindquantum.core.gates import RotPauliString
from mindquantum.core.gates import UnivMathGate
from mindspore import Tensor, ops
from mindquantum.core.circuit import UN
from mindquantum.core.operators import Hamiltonian             # 导入Hamiltonian模块，用于构建哈密顿量
from mindquantum.framework import MQLayer
from mindspore.nn import  TrainOneStepCell
from mindspore.nn import SoftmaxCrossEntropyWithLogits                         # 导入SoftmaxCrossEntropyWithLogits模块，用于定义损失函数
from mindspore.nn import Adam                                                  # 导入Adam模块用于定义优化参数
from mindspore.train import Accuracy, Model, LossMonitor                       # 导入Accuracy模块，用于评估预测准确率
import mindspore as ms
from mindspore.dataset import NumpySlicesDataset
from torch.utils.data import DataLoader# 导入NumpySlicesDataset模块，用于创建模型可以识别的数据集
import sys
sys.path.append('..')
from data_processing import X_train,X_test,y_train,y_test
num_layer = 3
# 定义标准差和形状
stddev = 0.02
shape_nnp = (num_layer, 8)
shape_stp = (num_layer, 12)

rtype = np.float64
ctype = np.complex128
# 使用 numpy 生成随机数矩阵
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(rtype)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(rtype)
#Operator Pool
unbound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[0] for i in range(8)]
bound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[1] for i in range(8,12)]

/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR


In [ ]:
softmax = ops.Softmax(axis=-1)

out = softmax(Tensor(stp, ms.float32))
out[0].shape

In [ ]:
TimeEvolution(QubitOperator(terms=unbound_opeartor_pool[0],coefficient=-0.213),-0.25).circuit

In [120]:
def DQAS_ansatz(Structure_p:np.array,Ansatz_p:np.array,n_layer:int,n_qbits:int=8):
    """
    和 DQAS 文章描述的一致，生成权重线路
    """
    stp = softmax(Tensor(Structure_p, ms.float32))
    ansatz = Circuit()
    for i in range(n_layer):
        
        paramertized_part_count=0
        for index_op,each_op in enumerate(unbound_opeartor_pool):

            ansatz_param = Ansatz_p[i,index_op]
            Structure_param =float(stp[i,index_op])
            ansatz += TimeEvolution(QubitOperator(terms=each_op,coefficient=ansatz_param),time=Structure_param).circuit
            paramertized_part_count+=1
            
        for index_op,each_op in enumerate(bound_opeartor_pool):
            #print(index_op,each_op)
            op = GroupedPauli('IZZYYXZI')
            tmp_cir = Circuit([GroupedPauli('IZZYYXZI').on(range(n_qbits))])
            matrix = tmp_cir.matrix()
            print(matrix.shape)
            ansatz += UnivMathGate(matrix_value=matrix*stp[i,index_op+paramertized_part_count],name=op.pauli_string).on(range(n_qbits))  
    
    
    #以下开始计算 loss
    # Encoder
    prg = PRGenerator('alpha')
    nqbits = n_qbits
    encoder = Circuit()
    encoder += UN(H, nqbits)                                  # H门作用在每1位量子比特
    for i in range(nqbits):                                   # i = 0, 1, 2, 3
        encoder += RY(prg.new()).on(i)                 # RZ(alpha_i)门作用在第i位量子比特
    encoder = encoder.no_grad()
    encoder = encoder.as_encoder()# Encoder作为整个量子神经网络的第一层，不用对编码线路中的梯度求导数，因此加入no_grad()
    
    hams = [Hamiltonian(QubitOperator(f'Z{i}')) for i in [0,1]]
    #ansatz = ansatz.as_ansatz()
    ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
    circuit = encoder+ ansatz.as_ansatz()    
    sim = Simulator('mqvector', n_qubits=nqbits)
    grad_ops = sim.get_expectation_with_grad(hams,
                                         circuit,
                                         parallel_worker=5)
    
    QuantumNet = MQLayer(circuit, hams, sim, grad_ops)
    loss_fn = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')    
    sum_loss = 0.0
    #统计所有数据的平均损失
    all_data = Tensor(X_train,ms.float32)
    all_label = Tensor(y_train,ms.int8)
    loss_value = loss_fn()
    
    
    
      
    
    
    
    
    
    
    
    
    
    
    
    
            
    

In [ ]:
ansatz = DQAS_ansatz(circuit=Circuit(),Structure_p=stp,Ansatz_p=nnp,n_layer=3)

In [ ]:
op = GroupedPauli(pauli_string='IZZYYXZI')
circ1 = Circuit([GroupedPauli('IZZYYXZI').on(range(8))])
matrix = circ1.matrix()
matrix.shape
# UnivMathGate(matrix_value=matrix*0.12,name=op.pauli_string).n_qubits

In [ ]:
circuit = Circuit()
circuit += RX(0.0).on(0)
circuit

In [ ]:
sim = Simulator(backend='mqvector', n_qubits=1)
sim.reset()
sim.apply_circuit(circuit=circuit)
sim.get_qs()

In [ ]:
import mindspore as ms
from mindspore import Tensor, ops, nn
from mindspore.ops import composite as C
import numpy as np

# 定义一个简单的函数
def f(x, y):
    return x**2 + 2*y

# 包装函数以便计算值和梯度
def value_and_grad(f, argnums=(0,)):
    def wrapper(*args):
        # 将输入转换为 MindSpore 的 Tensor
        ms_args = [Tensor(arg, dtype=ms.float32) for arg in args]
        
        # 定义计算梯度的函数
        def grad_fn(*ms_args):
            return f(*ms_args)
        
        # 计算函数值
        value = f(*ms_args)
        
        # 计算梯度
        grads = C.GradOperation(get_all=True, sens_param=False)(grad_fn)(*ms_args)
        
        # 提取指定参数的梯度
        selected_grads = [grads[i] for i in argnums]
        
        return value.asnumpy(), [g.asnumpy() for g in selected_grads]
    
    return wrapper

# 示例使用
g = value_and_grad(f)
value, grad = g(np.array(1.0), np.array(2.0))
print("Value:", value)  # 输出: 5.0
print("Grad:", grad)    # 输出: [2.0]

g = value_and_grad(f, argnums=(0, 1))
value, grads = g(np.array(1.0), np.array(2.0))
print("Value:", value)  # 输出: 5.0
print("Grads:", grads)  # 输出: [2.0, 2.0]

In [ ]:
import mindspore as ms
from mindspore import Tensor
from mindspore.nn import SoftmaxCrossEntropyWithLogits
import numpy as np

# 定义损失函数
loss_fn = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')

# 示例输入数据和标签
logits = Tensor(np.array([[1.0, 2.0], [1.0, 2.0]]), ms.float32)
labels = Tensor(np.array([2, 1]), ms.int32)

# 计算损失值
loss_value = loss_fn(logits, labels)

# 打印损失值
print("Loss value:", loss_value.asnumpy())